# AI Comic Book Generator - Colab Launcher

This notebook sets up the environment and runs the AI Comic Generator pipeline.
**Uses FREE Google Gemini API by default** (1500 requests/day).

In [ ]:
# @title 1. Setup Environment
# @markdown Enter your GitHub repo URL and branch.
REPO_URL = "https://github.com/yogesh-dixit-dev/comic-generator.git" # @param {type:"string"}
BRANCH = "master" # @param {type:"string"}

import os

if not os.path.exists("comic-gen"):
    !git clone -b $BRANCH $REPO_URL comic-gen
else:
    %cd comic-gen
    !git pull
    %cd ..

%cd comic-gen
!pip install -q -r requirements.txt
print("✅ Environment Set Up!")

In [ ]:
# @title 2. Configuration (FREE Gemini API)
# @markdown Get your FREE API key from: https://aistudio.google.com/apikey
import os
from google.colab import userdata

# Try getting from Colab Secrets first
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
    print("✅ Loaded API keys from Colab Secrets")
except:
    print("⚠️ No Colab Secrets found. You'll need to enter manually.")
    pass

# Manual override if needed
if not os.environ.get("GOOGLE_API_KEY"):
    print("📝 Get your FREE Gemini API key from: https://aistudio.google.com/apikey")
    os.environ["GOOGLE_API_KEY"] = input("Enter Google API Key: ")
if not os.environ.get("HF_TOKEN"):
    os.environ["HF_TOKEN"] = input("Enter Hugging Face Token: ")

In [ ]:
# @title 3. Run Pipeline
# @markdown Upload your story file to the 'comic-gen' folder using the file browser on the left.
INPUT_FILE = "story.txt" # @param {type:"string"}
OUTPUT_DIR = "output" # @param {type:"string"}
HF_REPO_ID = "your-username/comic-dataset" # @param {type:"string"}

# Make sure we're in the project directory
import os
os.chdir('/content/comic-gen')
print(f"📂 Working directory: {os.getcwd()}")

# Run the main script
!python src/main.py --input "$INPUT_FILE" --output "$OUTPUT_DIR" --colab --storage hf --hf_repo "$HF_REPO_ID"

In [ ]:
# @title 4. Download Output (Optional)
# @markdown If you didn't use HF storage, download the zip here.
from google.colab import files
import os

if os.path.exists('output'):
    !zip -r comic_output.zip output
    files.download('comic_output.zip')
    print("✅ Download started!")
else:
    print("⚠️ No output folder found. Did the pipeline run successfully?")